In [86]:
import random
from alive_progress import alive_bar, alive_it

Notes on bit manipulation in indices calculations of OctTrees:
- right shift = floor dividing the number by 2^shift (think if you did the same to a decimal number then it's divided by 10 essentially) - and vice versa for left shift 
- in bitwise AND operations, if one of the integers have a shorter bit length then it will be padded with 0s to the left to match the other operand's bit length

I think I'm thinking about the bit indexing a bit too much... in the sense that I'm overcomplicating WHY it's done a certain way. Okay. So from wiki, the to obtain the node index we get the different most significant bits of each channel (e.g. 4r + 2g + b). So... the thing that perplexes me a little is HOW the indexing works, which doesn't seem to be explained anywhere. But here are some notes to explain the things I was confused about:

The main thing that confused me a little bit, was why are we shifting each channel by a different amount? Surely that 
doesn't make sense from a calculation standpoint, because we want to look at the same magnitude bits at each level. Key idea is to understand that each bit represents which side of the 
subcube we stand at (because an Octree is essentially created by dividing each subcube's dimensions in half and then allocating children points to the different centres created, we can think of each bit as which half the point should be at at each level.) BUT. just thinking about it more simply, the different shift amounts is just because we want to create the end bits more easily. What we are doing here:
```
def compute_index(self, r, g, b, l):
    # what level of division are we looking at? - caps at 8 because value <= 255
    shift = 8 - l
    # Different shift values below are just to position the resulting bit in the correct position
    # in the final bitwise OR. 
    rc = r >> shift - 2 & 0x4
    gc = g >> shift - 1 & 0x2
    bc = b >> shift & 0x1
    return rc | gc | bc
```

is really the same as:
```
def computer_index(self, r, g, b, l):
    shift = 8-l 
    # This results in the same thing, where we first grab the bit of interest, then 
    # shift it to the desired position
    rc = (r >> shift & 0x1) <<  2
    gc = (g >> shift & 0x1) << 1
    bc = b >> shift & 0x1
    return rc | gc | bc
```

I think if I was more familiar with bit manipulation I wouldn't have thought in the wrong direction for so long... :( But good learning!
    

In [90]:
def compute_index(r, g, b, l):
    # what level of division are we looking at? - caps at 8 because value <= 255
    shift = 8 - l
    # Different shift values below are just to position the resulting bit in the correct position
    # in the final bitwise OR. 
    # If l has a possibility of exceeding 6, this version also acts as a control point 
    # because for example if l = 7, the r shift will be negative, which will raise an error.
    rc = r >> shift - 2 & 0x4
    gc = g >> shift - 1 & 0x2
    bc = b >> shift & 0x1
    return rc | gc | bc

def compute_index_2(r, g, b, l):
    shift = 8-l 
    # This results in the same thing, where we first grab the bit of interest, then 
    # shift it to the desired position
    rc = (r >> shift & 0x1) <<  2
    gc = (g >> shift & 0x1) << 1
    bc = b >> shift & 0x1
    return rc | gc | bc

#check that the two functions are equivalent 
n_points = 100_000
for _ in alive_it(range(n_points), force_tty=True):
    l  = random.randrange(7)
    r = random.randrange(256)
    g = random.randrange(256)
    b = random.randrange(256)
    assert compute_index(r, g, b, l) == compute_index_2(r, g, b, l)

|████████████████████████████████████████| 100000/100000 [100%] in 1.3s (78726.58/s)                    00000 [73%] in 1s (78466.7/s, eta: 0s) ▃▁▃ 89600/100000 [90%] in 1s (78563.3/s, eta: 0s)                 
